Evaluate Spacy models for person names, orgs and locations using the Presidio Evaluator framework

Data = `generated_test_November 12 2019`

In [ ]:
import spacy

from presidio_evaluator.evaluation import Evaluator, ModelError
from presidio_evaluator.data_generator import read_synth_dataset
%reload_ext autoreload
%autoreload 2



Select data for evaluation:

In [ ]:
synth_samples = read_synth_dataset("../../data/synth_dataset.txt")
print(len(synth_samples))
DATASET = synth_samples

In [ ]:
from collections import Counter
entity_counter = Counter()
for sample in DATASET:
    for span in sample.spans:
        entity_counter[span.entity_type]+=1

In [ ]:
entity_counter

In [ ]:
DATASET[1]

In [ ]:
#max length sentence
max([len(sample.tokens) for sample in DATASET])

Select models for evaluation:

In [ ]:
models = ["en_core_web_lg", "en_core_web_trf"]

Run evaluation on all models:

In [ ]:
from presidio_evaluator.models import SpacyModel


for model in models:
    print("-----------------------------------")
    print("Evaluating model {}".format(model))
    nlp = spacy.load(model)
    spacy_model = SpacyModel(model=nlp,entities_to_keep=['PERSON','GPE','ORG'])
    evaluator = Evaluator(model=spacy_model)
    evaluation_results = evaluator.evaluate_all(DATASET)
    scores = evaluator.calculate_score(evaluation_results)
    
    print("Confusion matrix:")
    print(scores.results)

    print("Precision and recall")
    scores.print()
    errors = scores.model_errors

Custom evaluation

In [ ]:
#evaluate custom sentences
nlp = spacy.load(spacy_ft_100)


### Results analysis

In [ ]:
#sent = input("Enter sentence: ")
sent = 'David is talking loudly'
doc = nlp(sent)
for ent in doc.ents:
    print("Entity = {} value = {}".format(ent.label_,ent.text))

#### False positives

1. Most false positive tokens:

In [ ]:
ModelError.most_common_fp_tokens(errors)

In [ ]:
fps_df = ModelError.get_fps_dataframe(errors,entity=['GPE'])
fps_df[['full_text','token','prediction']]

2. False negative examples

In [ ]:
errors = scores.model_errors
ModelError.most_common_fn_tokens(errors,n=50, entity=['PERSON'])

More FN analysis

In [ ]:
fns_df = ModelError.get_fns_dataframe(errors,entity=['GPE'])

In [ ]:
fns_df[['full_text','token','annotation','prediction']]

In [ ]:
print("All errors:\n")
[print(error,"\n") for error in errors]